## 整理程序

In [72]:
import os
import requests
import re
from bs4 import BeautifulSoup
import lxml
import os.path
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,WebDriverException
from selenium.webdriver.common.keys import Keys
import time
import warnings
warnings.filterwarnings("ignore")
import pickle
import pandas as pd
from requests.exceptions import ProxyError,ConnectionError,Timeout,HTTPError
import itertools
import random
from lxml import etree


# 设置请求头类
class GetHeaders(object):
    user_agent_list = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 "
        "(KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 "
        "(KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 "
        "(KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 "
        "(KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 "
        "(KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 "
        "(KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 "
        "(KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 "
        "(KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 "
        "(KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 "
        "(KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 "
        "(KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 "
        "(KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 "
        "(KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 "
        "(KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 "
        "(KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 "
        "(KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 "
        "(KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 "
        "(KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'
       ]

    def getHeaders(self):
        headers={
         "Accept": "application/json, text/javascript, */*; q=0.01",
         "Accept-Encoding": "gzip, deflate, br",
         "Accept-Language": "zh-CN,zh;q=0.8",
         "Connection": "keep-alive",
         "Content-Type":"application/x-www-form-urlencoded; charset=UTF-8",
         'User-Agent':random.choice(self.user_agent_list),
        }
        return headers

    
###############################################
##生成一个ip池
class Ip_pool():
    '''
    这是一个获取可用的ip池的类
    '''
    def __init__(self,ip_url_ls= ['http://www.xicidaili.com/nn/',
                                 'http://www.xicidaili.com/nt/',
                                 'http://www.xicidaili.com/wn/',
                                 'http://www.xicidaili.com/wt/']):
        self.ip_url_ls = ip_url_ls
        
        
    def check_chinese(self,check_str):
        zhPattern = re.compile(u'[\u4e00-\u9fa5]+')
        match = zhPattern.search(check_str)
        if match:
            return True
        else:
            return False
    
    def visit_url(self,ip_url,prox_ip=None):
        #headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.103 Safari/537.36"}
        headers = GetHeaders().getHeaders()
        print(headers)
        is_continue = True
        while is_continue:
            try:
                if prox_ip is None:
                    res = requests.get(ip_url,headers = headers)  #访问url，不设代理ip访问
                else:
                    http = prox_ip[0]
                    ip = http+"://"+prox_ip[1]
                    proxies={http:ip}
                    res = requests.get(self.ip_url,headers = headers,proxies = proxies) #设代理ip访问
                html =  res.content.decode('utf-8')
                soup  =  BeautifulSoup(html,'lxml')
                is_continue = False 
            
            except HTTPError as e:
                print("断网")
                time.sleep(2)

            except Timeout as e:
                print("请求超时")
                time.sleep(2)

            except ConnectionError as e:
                print("访问被拒")
                is_continue = True
                time.sleep(2)

        return soup


    #*******获取ip信息**********
    def get_iplist(self,soup):

        '''
        功能：对解析后的网页进行清洗，获得目标变量

        参数
        ----
        soup：BeautifulSoup对象，网页元素经过解析后的对象，包含了ip、端口号以及其他信息。

        返回值
        -----
        result:数组，装载了清洗得到的目标内容，如何ip、端口号以及其他信息
        '''
        result = []
        for i in soup.select("#ip_list .odd"):  #遍历每一组信息
            string = ""                         #初始化空字符
            for j in i.select("td"):           #信息存在td标签的text中
                if j.text.strip()!='':         #过滤掉没有text的td标签
                    info = j.text.strip()      #去除两边的空字符
                    string = string + info + "|"

            for m in i.select(".bar"):      #反应速度和连接时长都在此处
                time = m["title"]           #反应时间和连接时间
                string = string+time+"|"    #
            result.append(string)           #保存每一组的string
        return result

    def get_ip_form(self,result=None):
        '''
        功能：对获取的ip信息进行整合成符合requests的形式

        参数
        ----
        result：数组，装载了清洗得到的目标内容，如何ip、端口号以及其他信息

        返回值
        -----
        ip_list:数组，获取新的ip组

        '''
        ip_list = []                  #用来装整合的ip
        for i in result:              #遍历每一条ip信息
            if 'sock' not in i:       #删选socket类的代理ip
                a = i.split("|")      #分割
                type1 = a[4].lower() #将代理ip的类型全部变成小写
                ip = a[0]            #ip号
                port=a[1]            #代理ip端口
                ip_list.append([type1,type1+"://"+ip+":"+port]) #添加整合好的ip记录
        return ip_list

    def check_ip(self,test_url,ip_list=None,timeout=10):
        '''
        功能：对每一个ip进行测试，返回有效的，提除无效的

        参数
        ----
        ip_list：数组，装载代理ip的类型，代理ip和端口
        url：字符串，用来检测ip是否可用的url
        timeout:整型，访问时间上限，如果超过该时间限制仍访问不成功，即报错。

        返回值
        -----
        new_ip_list:数组，获取新的ip组

        '''
        new_ip_list = [] 
        for i in ip_list:
            proxies = {i[0]:i[1]}
            try:
                if '天' not in i[1]and not self.check_chinese(i[1]):
                    res = requests.get(test_url,proxies=proxies,timeout=timeout)
                    new_ip_list.append(i)
                    
            except Exception as e:
                print("坏ip:",i)
            
                
            except KeyboardInterrupt:
                return new_ip_list
        return new_ip_list

    def get_ip_pool(self,test_url="https://music.163.com/",timeout = 3):
        timeout = timeout
        ip_pool = []
        for ip_url in self.ip_url_ls:
            print(ip_url)
            soup = self.visit_url(ip_url) ##访问西刺网
            result = self.get_iplist(soup) #获取第一页的ip结果
            ip_list = self.get_ip_form(result) #获取ip形式
            new_ip_list = self.check_ip(test_url,ip_list,timeout) #检查ip
            ip_pool.extend(new_ip_list)
        return ip_pool